In [29]:
import requests
import pyotp
import pandas as pd
import time

In [39]:
CLIENT_CODE = ""
MPIN = ""
TOTP_KEY = ""
BASE_URL = "https://tradingorestapi.swastika.co.in"
SLEEP_TIME = 1

gtt_orders_df = pd.read_excel("GTT_ORDERS.xlsx", index_col="SN")

In [40]:
totp = pyotp.TOTP(TOTP_KEY)
session = requests.Session()
session.headers['Content-Type'] = 'application/json'


def login():
    time.sleep(SLEEP_TIME)
    endpoint = "/auth/TOTP/VerifyTotp"
    payload = {"Totp": totp.now(),
               "ClientCode": CLIENT_CODE,
               "MPIN": MPIN,
               "GenerationSourceTP": "RAJTP",
               "IPAddress": "192.168.10"}
    
    response = session.post(BASE_URL+endpoint, json=payload).json()
    
    if response["StatusCode"] == 200:
        session.headers['Authorization'] = f'Bearer {response["Result"]["Data"]["AccessToken"]}'
        print("login success")
        return True
    
    print(response)
    return False


def get_gtt_ids():
    """
    Function to get all the ids of active gtt orders
    """
    time.sleep(SLEEP_TIME)
    endpoint = "/kb/GTTOrders/GetPendingGTTOrder"
    payload = {"Uid": CLIENT_CODE}
    response = session.post(BASE_URL+endpoint, json=payload).json()
#     return response
    if response["StatusCode"] == 200:
        return [i["Al_id"] for i in response["Result"]["Data"]]
    print(response)
    return []


def place_gtt_order(symbol, order_type, quantity, price, trigger_price, exchange="NSE", trigger_dir=None, **kwargs):
    """
    Function to place the GTT order
    """
    time.sleep(SLEEP_TIME)
    endpoint = "/kb/GTTOrders/PlaceGTTOrder"

    if not trigger_dir:
        if order_type.lower() == "sell":
            trigger_dir = "LTP_A_O"
        else:
            trigger_dir = "LTP_B_O"
    
    payload = {
                "Uid": str(CLIENT_CODE),
                "Actid": str(CLIENT_CODE),
                "Tsym": str(symbol),
                "Exch": str(exchange),
                "Ai_t": str(trigger_dir),
                "Validity": "GTT",
                "D": str(trigger_price),
                "Remarks": kwargs.pop("remarks", "RAJTP"),
                "Trantype": order_type,
                "Prctyp": kwargs.pop("prc_type", "MKT"),
                "Prd": "C",
                "Ret": kwargs.pop("RET", "DAY"),
                "Qty": str(quantity),
                "Prc": str(price),
            }
    
    response = session.post(BASE_URL+endpoint, json=payload).json()
    
    if response["StatusCode"] == 200:
        print(response["Result"]["ResponseMessage"], f"for {symbol}")
        return response["Result"]["Data"]["Al_id"]
    
    print(response)
    return False


def delete_gtt_orders(gtt_id):
    """
    Function to delete GTT orders
    """
    time.sleep(SLEEP_TIME)
    endpoint = "/kb/GTTOrders/CancelGTTOrder"
    payload = {
                 "Uid": CLIENT_CODE,
                 "Al_id": str(gtt_id)
              }
    response = session.post(BASE_URL+endpoint, json=payload).json()
    if response["StatusCode"] != 200:
        print(response)
        return False
    print("Deleted ", gtt_id)
    return response

login()

login success


True

In [41]:
# delete existing GTT orders..
_ = [delete_gtt_orders(sym) for sym in get_gtt_ids()]

print("\n\n")

# Place new gtt orders..
order_ids = []
for row in gtt_orders_df.itertuples():
    aid = place_gtt_order(symbol=row.SYMBOL, order_type=row.ORDER_TYPE, quantity=row.QUANTITY, price=row.PRICE,
                          trigger_price=row.TRIGGER_PRICE, exchange=row.EXCHANGE)
    order_ids.append(aid)

Deleted 
Deleted 
Deleted 
Deleted 
Order placed successfully for YESBANK-EQ
Order placed successfully for COFORGE-EQ
Order placed successfully for HDFCBANK-EQ
Order placed successfully for SBIN-EQ
